In [1]:
import networkx as nx
# Example 1: small directed cycle
G1 = nx.DiGraph()
G1.add_edges_from([
    ("A", "B"),
    ("B", "C"),
    ("C", "A"),
])

# Example 2: directed acyclic graph (DAG)
G2 = nx.DiGraph()
G2.add_edges_from([
    ("1", "2"),
    ("1", "3"),
    ("2", "4"),
    ("3", "4"),
])

def print_directed_and_transpose(G, name: str):
    GT = G.reverse(copy=True)
    print(f"=== {name} ===")
    print("Original edges:")
    print(sorted(G.edges()))
    print("Transposed edges:")
    print(sorted(GT.edges()))
    print()

print_directed_and_transpose(G1, "Directed Graph G1 (cycle)")
print_directed_and_transpose(G2, "Directed Graph G2 (DAG)")


=== Directed Graph G1 (cycle) ===
Original edges:
[('A', 'B'), ('B', 'C'), ('C', 'A')]
Transposed edges:
[('A', 'C'), ('B', 'A'), ('C', 'B')]

=== Directed Graph G2 (DAG) ===
Original edges:
[('1', '2'), ('1', '3'), ('2', '4'), ('3', '4')]
Transposed edges:
[('2', '1'), ('3', '1'), ('4', '2'), ('4', '3')]



In [2]:
# Problem 1.2  Undirected graphs and their inverse (complement)

# Example 1: a simple path
H1 = nx.Graph()
H1.add_edges_from([
    ("A", "B"),
    ("B", "C"),
    ("C", "D"),
])

# Example 2: a small triangle plus an isolated vertex
H2 = nx.Graph()
H2.add_nodes_from(["1", "2", "3", "4"])  # add all vertices explicitly
H2.add_edges_from([
    ("1", "2"),
    ("2", "3"),
    ("1", "3"),
])  # "4" is isolated

def print_undirected_and_inverse(H, name: str):
    H_inv = nx.complement(H)
    print(f"=== {name} ===")
    print("Original edges:")
    print(sorted(map(tuple, map(sorted, H.edges()))))
    print("Inverse (complement) edges:")
    print(sorted(map(tuple, map(sorted, H_inv.edges()))))
    print()
print_undirected_and_inverse(H1, "Undirected Graph H1 (path)")
print_undirected_and_inverse(H2, "Undirected Graph H2 (triangle + isolated)")


=== Undirected Graph H1 (path) ===
Original edges:
[('A', 'B'), ('B', 'C'), ('C', 'D')]
Inverse (complement) edges:
[('A', 'C'), ('A', 'D'), ('B', 'D')]

=== Undirected Graph H2 (triangle + isolated) ===
Original edges:
[('1', '2'), ('1', '3'), ('2', '3')]
Inverse (complement) edges:
[('1', '4'), ('2', '4'), ('3', '4')]



In [3]:
# Problem 1.4: Simple planar graphs (for duals) – purely descriptive

# Example: triangle graph K3
K3 = nx.Graph()
K3.add_edges_from([
    ("A", "B"),
    ("B", "C"),
    ("C", "A"),
])

print("Planar graph K3 (triangle) – edges:", sorted(map(tuple, map(sorted, K3.edges()))))

print(
    "Dual of K3 (conceptually): vertices = {F_inside, F_outside}, "
    "edges = 3 parallel edges between them."
)


Planar graph K3 (triangle) – edges: [('A', 'B'), ('A', 'C'), ('B', 'C')]
Dual of K3 (conceptually): vertices = {F_inside, F_outside}, edges = 3 parallel edges between them.


In [4]:
# Problem 1.5: Example of a non-planar graph (K3,3)

K33 = nx.Graph()
left = ["u1", "u2", "u3"]
right = ["v1", "v2", "v3"]

for u in left:
    for v in right:
        K33.add_edge(u, v)

print("Non-planar graph K3,3 – edges:")
print(sorted(map(tuple, map(sorted, K33.edges()))))

# networkx has a planarity test
is_planar, cert = nx.check_planarity(K33)
print("Is K3,3 planar?", is_planar)
print("Since it is not planar, a planar dual is not well-defined.")


Non-planar graph K3,3 – edges:
[('u1', 'v1'), ('u1', 'v2'), ('u1', 'v3'), ('u2', 'v1'), ('u2', 'v2'), ('u2', 'v3'), ('u3', 'v1'), ('u3', 'v2'), ('u3', 'v3')]
Is K3,3 planar? False
Since it is not planar, a planar dual is not well-defined.


In [5]:
#  Graph definition for Problem 2

graph = {
    "A": ["B", "C"],
    "B": ["A", "C"],
    "C": ["A", "B", "D"],
    "D": ["C"],
}
V = set(graph.keys())
V


{'A', 'B', 'C', 'D'}

In [6]:
# Bron–Kerbosch without pivoting, with detailed trace

def bron_kerbosch(R, P, X, graph, depth=0, cliques=None):
    if cliques is None:
        cliques = []

    indent = "  " * depth
    print(f"{indent}BK call: R={sorted(R)}, P={sorted(P)}, X={sorted(X)}")

    if not P and not X:
        # R is a maximal clique
        print(f"{indent}--> report maximal clique: {sorted(R)}")
        cliques.append(R.copy())
        return cliques

    # We iterate over a snapshot of P because we modify P during the loop
    for v in list(P):
        print(f"{indent}  Considering vertex v={v}")
        # Neighbours of v
        N_v = set(graph[v])

        # Recursive call with v added to R, and P/X intersected with N(v)
        bron_kerbosch(
            R | {v},
            P & N_v,
            X & N_v,
            graph,
            depth=depth + 1,
            cliques=cliques
        )
        # Move v from P to X
        P.remove(v)
        X.add(v)
        print(f"{indent}  Move v={v} from P to X -> P={sorted(P)}, X={sorted(X)}")

    return cliques

# Initial call:
R0 = set()
P0 = V.copy()   # {A, B, C, D}
X0 = set()

all_cliques = bron_kerbosch(R0, P0, X0, graph)
print("\nAll maximal cliques found:", [sorted(C) for C in all_cliques])


BK call: R=[], P=['A', 'B', 'C', 'D'], X=[]
  Considering vertex v=C
  BK call: R=['C'], P=['A', 'B', 'D'], X=[]
    Considering vertex v=B
    BK call: R=['B', 'C'], P=['A'], X=[]
      Considering vertex v=A
      BK call: R=['A', 'B', 'C'], P=[], X=[]
      --> report maximal clique: ['A', 'B', 'C']
      Move v=A from P to X -> P=[], X=['A']
    Move v=B from P to X -> P=['A', 'D'], X=['B']
    Considering vertex v=A
    BK call: R=['A', 'C'], P=[], X=['B']
    Move v=A from P to X -> P=['D'], X=['A', 'B']
    Considering vertex v=D
    BK call: R=['C', 'D'], P=[], X=[]
    --> report maximal clique: ['C', 'D']
    Move v=D from P to X -> P=[], X=['A', 'B', 'D']
  Move v=C from P to X -> P=['A', 'B', 'D'], X=['C']
  Considering vertex v=B
  BK call: R=['B'], P=['A'], X=['C']
    Considering vertex v=A
    BK call: R=['A', 'B'], P=[], X=['C']
    Move v=A from P to X -> P=[], X=['A', 'C']
  Move v=B from P to X -> P=['A', 'D'], X=['B', 'C']
  Considering vertex v=A
  BK call: R=['A'

In [7]:
#Post-processing — list maximal and maximum cliques

def clique_size(clique):
    return len(clique)

sorted_cliques = sorted(all_cliques, key=lambda C: (-len(C), sorted(C)))
print("Maximal cliques (sorted by size):")
for C in sorted_cliques:
    print(f"  clique {sorted(C)}, size={len(C)}")

max_size = max(len(C) for C in sorted_cliques)
maximum_cliques = [C for C in sorted_cliques if len(C) == max_size]

print("\nMaximum clique size:", max_size)
print("Maximum cliques:")
for C in maximum_cliques:
    print(" ", sorted(C))


Maximal cliques (sorted by size):
  clique ['A', 'B', 'C'], size=3
  clique ['C', 'D'], size=2

Maximum clique size: 3
Maximum cliques:
  ['A', 'B', 'C']
